# Setup

## Imports

In [ ]:
import sys
import icecream
sys.path.append('./translator')

In [ ]:
import torch
import rp
from translator.trainer import MUNIT_Trainer as Trainer
from translator.data import ImageFolder

In [ ]:
from IPython.display import clear_output

## Other Setup

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Functions

In [ ]:
#Alphablock
config_file       = './translator/configs/alphablock_without_ssim_256.yaml'
image_folder_path = './datasets/alphacube/scenes/'
# image_folder_path = './datasets/alphacube/anim_2/'
# image_folder_path = './datasets/alphacube/anim_1/'
image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/alphabetcube_L/SyntheticData/Anim3/Renderings'
checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints'
# checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints/old_checkpoints/v0.0.7'
# checkpoint_folder = './translator/save'

In [ ]:
#Sunkist
config_file       = './translator/configs/config.yaml'
image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/sunkist/synthetic/RenderingsAnim'
checkpoint_folder = './translator/trained_models/outputs/config/checkpoints'

In [ ]:
config = rp.load_yaml_file(config_file)

In [ ]:
trainer=Trainer(config, trainable=False).to(device)

In [ ]:
trainer.resume(checkpoint_folder, config)

In [ ]:
rp.display_image(rp.tiled_images(rp.as_numpy_images(trainer.texture_pack())))

## TODO: Make the Trainer save the neural texture once, so during inference we don't need to run it over and over again (it's redundant to do that seeing as the texture never changes)

In [ ]:
aug = {}
aug["new_size_min"] = config["new_size_min_a"]
aug["new_size_max"] = config["new_size_max_a"]
aug["output_size" ] = (-1,-1) #Meaningless when skip_crop = True
# aug["output_size" ] = (320,320) #Meaningless when skip_crop = True
image_folder = ImageFolder(root=image_folder_path, precise=True, augmentation=aug)
image_folder.skip_crop = True

## Test 1

In [ ]:
o=rp.random_element(image_folder)

In [ ]:
_, height, width = o.shape
icecream.ic(height,width)

In [ ]:
rp.display_image(rp.as_numpy_image(o))

In [ ]:
h=o[None].to(device)
i=trainer.sample_a2b(h)


rp.display_image(rp.as_numpy_image(h[0]))
rp.display_image(rp.as_numpy_image(i[0]))

In [ ]:
i=trainer.sample(h,h)[:7]
for x in i:
    rp.display_image(rp.as_numpy_image(x[0]))

## Test 2

In [ ]:
def translate(image):
    #Input image is a UV-L Scene
    assert rp.is_image(image)
    image = rp.as_rgb_image  (image)
    image = rp.as_float_image(image)
    
    image = rp.as_torch_image(image)[None] #BCHW
    image = image.to(device)
    
    with torch.no_grad():
        output = trainer.sample_a2b(image)
    output = output[0]
    output = rp.as_numpy_image(output)
    
    return output

In [ ]:
images = rp.load_images(image_folder_path, show_progress=True)
clear_output()

images = [rp.cv_resize_image(image, (height, width), 'nearest') for image in images]
uvl_scenes=images.copy()

In [ ]:
import numpy as np
bad_indices=[i for i,x in enumerate(images) if not np.any(x)]
for i in bad_indices:
    del images[i]
    del uvl_scenes[i]

In [ ]:
display_eta = rp.eta(len(images))
translations = []
for index,image in enumerate(images):
    display_eta(index)
    with torch.no_grad():
        translations.append(translate(image))
clear_output()

In [ ]:
#Please be patient, this step can take a long time to complete (up to 5 minutes)
#This is because it's extremely inefficient lol. All images in this animation are being converted into base-64 in html
# rp.display_image_slideshow(translations[::5])

In [ ]:
video_path = 'untracked/uvl_animation.mp4'
rp.save_video_mp4([rp.as_rgb_image(rp.as_byte_image(x)) for x in images],video_path,video_bitrate='medium')
clear_output()
# translations=[rp.as_byte_image(x) for x in translations]
# translations=rp.as_numpy_array(translations)
# write_mp4(video_path, translations)

from IPython.display import Video
icecream.ic(video_path)
Video(video_path)

In [ ]:
video_path = 'untracked/animation.mp4'
rp.save_video_mp4(translations,'untracked/animation.mp4', video_bitrate='large')
clear_output()
# translations=[rp.as_byte_image(x) for x in translations]
# translations=rp.as_numpy_array(translations)
# write_mp4(video_path, translations)

from IPython.display import Video
icecream.ic(video_path)
Video(video_path)

In [ ]:
from source.unprojector import unproject_translations
from source.unprojector import unproject_translations_individually
from source.unprojector import combine_individual_unprojections
from source.scene_reader import extract_scene_uvs_and_scene_labels

In [ ]:
scene_uvs, scene_labels = extract_scene_uvs_and_scene_labels(rp.as_torch_images(rp.as_numpy_array(uvl_scenes)),[0,255])
translations=rp.as_numpy_array(translations)

In [ ]:
scene_trans = rp.as_torch_images(rp.as_numpy_array(translations))

height=min(scene_trans.shape[2],scene_uvs.shape[2])
width =min(scene_trans.shape[3],scene_uvs.shape[3])
scene_trans=scene_trans[:,:,:height,:width]
scene_uvs=scene_uvs[:,:,:height,:width]
scene_labels=scene_labels[:,:height,:width]


icecream.ic(scene_trans.shape, scene_trans.dtype, scene_trans.device);
icecream.ic(scene_trans.shape, scene_uvs.shape, scene_labels.shape);

In [ ]:
import gc
gc.collect()
recovery_resolution=1024
with torch.no_grad():
    recovered_textures, recovered_weights = unproject_translations_individually(scene_trans[::4],
                                                                   scene_uvs[::4],  scene_labels[::4],
                                                                   num_labels   =3                                        ,
                                                                   output_height=recovery_resolution                      ,
                                                                   output_width =recovery_resolution                      )
    
recovered_textures, recovered_weights =combine_individual_unprojections(recovered_textures, recovered_weights)

In [ ]:
# recovered_textures=(recovered_weights[:,:,None]*recovered_textures).sum(0)/(recovered_weights.sum(0)+.01)

In [ ]:
rp.display_image(rp.tiled_images(rp.as_numpy_images(recovered_textures)))

In [ ]:
from source.projector import project_textures

In [ ]:
reprojections=project_textures(scene_uvs,scene_labels,recovered_textures)

In [ ]:
rp.save_video(rp.as_numpy_images(reprojections),'untracked/reproject.mp4')
clear_output()
Video('untracked/reproject.mp4')

In [ ]:
import numpy as np
shadows = rp.as_numpy_array(translations)-rp.as_numpy_images(reprojections)
rp.save_video(.5+rp.as_numpy_images(shadows),'untracked/shadows.mp4')
clear_output()
Video('untracked/shadows.mp4')

In [ ]:
import numpy as np
shadows =rp.as_numpy_array(translations)-rp.as_numpy_images(reprojections)
# shadows = [rp.cv_gauss_blur(x,3) for x in shadows]
# shadows = rp.as_numpy_array(shadows)
# shadows_amp = np.abs(shadows)
# shadows_sign = np.sign(shadows)
# shadows = [rp.cv_dilate(rp.cv_erode(x,3),3) for x in shadows_amp]
# shadows = shadows * shadows_sign
shadows = rp.as_numpy_array(shadows)
rp.save_video(.5+rp.as_numpy_images(shadows),'untracked/shadows.mp4')
clear_output()
Video('untracked/shadows.mp4')

In [ ]:
import numpy as np
redone = shadows*3+rp.as_numpy_images(reprojections)
rp.save_video(rp.as_numpy_images(redone),'untracked/redone.mp4')
clear_output()
Video('untracked/redone.mp4')